In [10]:
import csv
data = {}

with open('data.csv') as csvfile:
  next(csvfile) # Skip header row
  spamreader = csv.reader(csvfile)
  for row in spamreader:
    if row[1] == 'None':
      continue
    data[row[0]] = row[1:]
print(data)
  

{'s1': ['4', '10', '7'], 's2': ['4', '8', '10'], 's3': ['1', '5', '3'], 's4': ['1', '2', '10'], 's5': ['1', '7', '4'], 's6': ['5', '3', '2'], 's7': ['7', '4', '3'], 's8': ['7', '8', '3'], 's9': ['6', '4', '1'], 's10': ['6', '4', '5'], 's11': ['6', '4', '3'], 's12': ['6', '4', '3'], 's13': ['6', '5', '7'], 's14': ['3', '7', '4'], 's15': ['3', '7', '4'], 's16': ['3', '7', '4'], 's17': ['3', '7', '4'], 's18': ['3', '5', '1'], 's19': ['3', '4', '1'], 's20': ['3', '4', '1'], 's21': ['3', '4', '10'], 's22': ['10', '8', '3'], 's23': ['10', '1', '3'], 's24': ['10', '8', '3'], 's25': ['10', '1', '3'], 's26': ['10', '3', '6'], 's27': ['10', '6', '4'], 's28': ['2', '1', '3'], 's29': ['2', '1', '3'], 's30': ['2', '1', '3']}


In [17]:
from collections import defaultdict, Counter
import pandas as pd

def create_optimized_groups(data):
    # Initialize a set to track which papers have been assigned to a group
    paper_assigned = set()
    
    # Initialize a dictionary to hold the groups
    groups = defaultdict(list)
    
    # Count preferences for each paper
    all_preferences = data[['First_Preference', 'Second_Preference', 'Third_Preference']].values.flatten()
    paper_preferences = Counter(paper for paper in all_preferences if pd.notna(paper))
    
    # Sort papers by popularity (most preferred first)
    sorted_papers = sorted(paper_preferences, key=paper_preferences.get, reverse=True)
    
    # Helper function to try adding a student to a group
    def try_add_student_to_group(student, prefs):
        for pref in prefs:
            if pd.notna(pref) and len(groups[pref]) < 4:  # Check for valid preference and group capacity
                groups[pref].append(student)
                return True
        return False

    # Step 1: Assign students with preferences to groups
    for _, row in data.iterrows():
        student = row['Name']
        prefs = [row['First_Preference'], row['Second_Preference'], row['Third_Preference']]
        
        # Attempt to assign the student based on their ranked preferences
        if not try_add_student_to_group(student, prefs):
            continue  # If no preferences are assignable, they will be handled in the next step

    # Step 2: Assign students with no preferences to any available group
    for _, row in data.iterrows():
        student = row['Name']
        prefs = [row['First_Preference'], row['Second_Preference'], row['Third_Preference']]
        
        # Check if the student was already assigned
        assigned = any(student in group for group in groups.values())
        
        if not assigned:
            # Assign to any group with space available
            for paper in sorted_papers:
                if len(groups[paper]) < 4:
                    groups[paper].append(student)
                    break

    # Step 3: Mark papers as assigned if they have groups
    for group in groups:
        if len(groups[group]) > 0:
            paper_assigned.add(group)
    
    return groups

# Example usage:
# Load the data (replace with your actual file path)
data = pd.read_csv('data.csv')

# Create optimized groups based on preferences
optimized_groups = create_optimized_groups(data)

# Display the optimized groups and their assigned papers
for group, members in optimized_groups.items():
    print(f"Group assigned to Paper {group}: {members}")


Group assigned to Paper 4.0: ['s1', 's2', 's19', 's20']
Group assigned to Paper 1.0: ['s3', 's4', 's5', 's25']
Group assigned to Paper 5.0: ['s6', 's13', 's18', 's31']
Group assigned to Paper 7.0: ['s7', 's8', 's26', 's27']
Group assigned to Paper 6.0: ['s9', 's10', 's11', 's12']
Group assigned to Paper 3.0: ['s14', 's15', 's16', 's17']
Group assigned to Paper 10.0: ['s21', 's22', 's23', 's24']
Group assigned to Paper 2.0: ['s28', 's29', 's30', 's32']
